In [1]:
from llama import Llama

In [2]:
from llama.tokenizer import Tokenizer
from llama.watermark import AaronsonWatermarker

In [3]:
tokenizer_path='tokenizer.model'
hashing_schema='lefthash'
tokenizer = Tokenizer(model_path=tokenizer_path)

In [4]:
watermarker = AaronsonWatermarker(
    vocab_size = tokenizer.n_words,
    tokenizer = tokenizer,
    hashing_schema = hashing_schema,
)

In [7]:

import json


input_file_path = "chat_output_aaronson.json"

input_file_path = "chat_output_none.json"


for input_file_path in ["chat_output_none.json","chat_output_aaronson.json"]:
    # 打开 JSON 文件并解析数据
    with open(input_file_path, "r") as input_file:
        output_list = json.load(input_file)

    # 循环遍历输出列表中的每个字典
    for item in output_list[:]:
        output=item['result']['generation']['content']
        test = watermarker.detect(output)
        print ("p-value:",test[0], "ST (null is 1):", test[1], "Z-score", test[2])
    
    print('\n\n')
                


            

p-value: 0.28473160574059664 ST (null is 1): 1.0328970306294418 Z-score 0.5688422322273254
p-value: 0.6362183598978113 ST (null is 1): 0.9100515365600585 Z-score -0.34836873412132263
p-value: 0.9416723506131344 ST (null is 1): 0.9292654642244664 Z-score -1.568968415260315
p-value: 0.2602087082006941 ST (null is 1): 1.028975261905329 Z-score 0.6427021026611328
p-value: 0.6203690724079558 ST (null is 1): 0.9844822810246394 Z-score -0.3064502477645874
p-value: 0.7521868864236793 ST (null is 1): 0.8712297848292759 Z-score -0.6813876628875732
p-value: 0.49673427102817624 ST (null is 1): 1.000732177734375 Z-score 0.00818606000393629
p-value: 0.06320135453408882 ST (null is 1): 1.0699093232094992 Z-score 1.5284425020217896
p-value: 0.2585317984113834 ST (null is 1): 1.0292382638469004 Z-score 0.6478784084320068
p-value: 0.46376818603564907 ST (null is 1): 1.0076051725374235 Z-score 0.09094490110874176
p-value: 0.5592421097572431 ST (null is 1): 0.9913367194098395 Z-score -0.14904797077178955


KeyboardInterrupt: 

In [25]:
import json
import pandas as pd
import numpy as np

# 假设您已经定义了watermarker.detect函数

none_p_values = []  # 存储 "none" 文件的p-value数据
aaronson_p_values = []  # 存储 "aaronson" 文件的p-value数据

for input_file_path in ["chat_output_none.json", "chat_output_aaronson.json"]:
    with open(input_file_path, "r") as input_file:
        output_list = json.load(input_file)

    for item in output_list[:100]:
        output = item['result']['generation']['content']
        test = watermarker.detect(output)
        p_value, _, _ = test  # 解包p-value
        
        if "none" in input_file_path:
            none_p_values.append(p_value)
        elif "aaronson" in input_file_path:
            aaronson_p_values.append(p_value)

# 取对数并创建DataFrame
p_values_df = pd.DataFrame({'none': none_p_values, 'aaronson': aaronson_p_values})
p_values_df['none_log'] = np.log(p_values_df['none'])
p_values_df['aaronson_log'] = np.log(p_values_df['aaronson'])

# 计算对数平均值
none_log_avg = p_values_df['none_log'].mean()
aaronson_log_avg = p_values_df['aaronson_log'].mean()

# 打印对数平均值
print("Log Average (none):", none_log_avg)
print("Log Average (aaronson):", aaronson_log_avg)

# 打印DataFrame
print(p_values_df)

# 打印对数列的平均值
print("Log Column Average (none_log):", p_values_df['none_log'].mean())
print("Log Column Average (aaronson_log):", p_values_df['aaronson_log'].mean())


Log Average (none): -1.2005159350580363
Log Average (aaronson): -inf
        none      aaronson  none_log  aaronson_log
0   0.284732  3.705700e-08 -1.256208    -17.110808
1   0.636218  0.000000e+00 -0.452213          -inf
2   0.941672  3.528497e-01 -0.060098     -1.041713
3   0.260209  0.000000e+00 -1.346271          -inf
4   0.620369  1.186196e-11 -0.477441    -25.157685
..       ...           ...       ...           ...
95  0.697958  3.597284e-01 -0.359596     -1.022406
96  0.292079  4.887499e-05 -1.230732     -9.926245
97  0.134136  6.308691e-03 -2.008899     -5.065827
98  0.333014  3.330139e-01 -1.099571     -1.099571
99  0.181666  1.059443e-01 -1.705586     -2.244842

[100 rows x 4 columns]
Log Column Average (none_log): -1.2005159350580363
Log Column Average (aaronson_log): -inf


/root/miniconda3/envs/distil/lib/python3.9/site-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [26]:
p_values_df['aaronson_log']

,none,aaronson,none_log,aaronson_log
0,0.284732,3.705700e-08,-1.256208,-17.110808
1,0.636218,0.000000e+00,-0.452213,-inf
2,0.941672,3.528497e-01,-0.060098,-1.041713
3,0.260209,0.000000e+00,-1.346271,-inf
4,0.620369,1.186196e-11,-0.477441,-25.157685
...,...,...,...,...
95,0.697958,3.597284e-01,-0.359596,-1.022406
96,0.292079,4.887499e-05,-1.230732,-9.926245
97,0.134136,6.308691e-03,-2.008899,-5.065827
98,0.333014,3.330139e-01,-1.099571,-1.099571


In [28]:
# 假设您已经创建了p_values_df DataFrame

count_below_minus_2 = (p_values_df['aaronson_log'] < -10).sum()
print("Number of values below -2:", count_below_minus_2)


Number of values below -2: 37


In [20]:
# 计算平均值
mean_none = p_values_df['none'].mean()
mean_aaronson = p_values_df['aaronson'].mean()

# 打印平均值
print("Mean p-value for 'none':", mean_none)
print("Mean p-value for 'aaronson':", mean_aaronson)

# 取对数
none_p_values_log = np.log(none_p_values)
aaronson_p_values_log = np.log(aaronson_p_values)

# 计算对数平均值
none_p_values_log_mean = np.mean(none_p_values_log)
aaronson_p_values_log_mean = np.mean(aaronson_p_values_log)

# 打印结果
print("Log Average for 'none':", log_avg_none)
print("Log Average for 'aaronson':", log_avg_aaronson)


Mean p-value for 'none': 0.47776225932931693
Mean p-value for 'aaronson': 0.04105624134949313
Log Average for 'none': 0.3756813067004241
Log Average for 'aaronson': 0.03590378136441573


/root/autodl-tmp/ckpt/ipykernel_973028/745771594.py:11: RuntimeWarning: divide by zero encountered in log
  aaronson_p_values_log = np.log(aaronson_p_values)
